In [169]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.layers import TextVectorization,Embedding,Dense,LSTM,Flatten,Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import gc

In [2]:
train_essays = pd.read_csv('./llm-detect-ai-generated-text/train_essays.csv')
train_prompts = pd.read_csv('./llm-detect-ai-generated-text/train_prompts.csv') 
test_essays = pd.read_csv('./llm-detect-ai-generated-text/test_essays.csv')

In [3]:
train_essays

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [4]:
train_essays.drop(['id','prompt_id'],axis=1,inplace=True)

In [5]:
labels = train_essays['generated']

In [6]:
train = train_essays.drop(['generated'],axis=1)

In [8]:
vocab = 16355

In [9]:
vec_layer = TextVectorization(max_tokens=vocab,output_mode='int',output_sequence_length=250)
full_text_array = train.to_numpy()

2023-11-19 12:10:53.742776: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-11-19 12:10:53.742806: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-11-19 12:10:53.742815: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-11-19 12:10:53.743323: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-19 12:10:53.743810: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
vec_layer.adapt(x_train)
def vectorize_text(train):
  # train = tf.expand_dims(train, -1)
  return vec_layer(train)
train = vectorize_text(train)

2023-11-19 12:10:55.012935: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [19]:
x_train,x_test,y_train,y_test = train_test_split(train,labels.to_numpy())

In [34]:
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [172]:
callbacks = [TensorBoard(log_dir='./logs',histogram_freq=1,update_freq=1),EarlyStopping(monitor='acc',patience=0)]

In [35]:
model = Sequential()
# model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(Embedding(16356,50))
# model.add(LSTM(128))
# model.add(LSTM(22))
# model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))


model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

history = model.fit(x_train,y_train,epochs=20,callbacks=callbacks)

Epoch 1/20


2023-11-19 12:30:37.729886: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 3s 33ms/step - loss: 0.6285 - acc: 0.9176
Epoch 2/20
33/33 [==============================] - 1s 22ms/step - loss: 0.4772 - acc: 0.9974
Epoch 3/20
33/33 [==============================] - 1s 23ms/step - loss: 0.3253 - acc: 0.9981
Epoch 4/20
33/33 [==============================] - 1s 21ms/step - loss: 0.2090 - acc: 0.9982
Epoch 5/20
33/33 [==============================] - 1s 21ms/step - loss: 0.1367 - acc: 0.9982
Epoch 6/20
33/33 [==============================] - 1s 20ms/step - loss: 0.0948 - acc: 0.9982
Epoch 7/20
33/33 [==============================] - 1s 19ms/step - loss: 0.0701 - acc: 0.9982
Epoch 8/20
33/33 [==============================] - 1s 17ms/step - loss: 0.0546 - acc: 0.9982
Epoch 9/20
33/33 [==============================] - 1s 19ms/step - loss: 0.0445 - acc: 0.9982
Epoch 10/20
33/33 [==============================] - 1s 19ms/step - loss: 0.0374 - acc: 0.9982
Epoch 11/20
33/33 [==============================] - 1s 18ms/step - lo

In [38]:
model.evaluate(x_test,y_test)

 6/11 [===============>..............] - ETA: 0s - loss: 0.0161 - acc: 0.9994

2023-11-19 12:36:49.774433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 12ms/step - loss: 0.0301 - acc: 0.9965


[0.030111832544207573, 0.9965217709541321]

In [81]:
new_data = pd.read_csv('train_drcat_04.csv')

In [82]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   essay_id  44206 non-null  object
 1   text      44206 non-null  object
 2   label     44206 non-null  int64 
 3   source    44206 non-null  object
 4   prompt    12911 non-null  object
 5   fold      44206 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [83]:
new_data.drop(['essay_id','source','prompt','fold'],axis=1,inplace=True)

In [84]:
new_data

,text,label
0,"In recent years, technology has had a profoun...",1
1,Should students participate in an extracurricu...,0
2,The electoral college is a symbol of mockery a...,0
3,This is why I think the principle should allow...,0
4,I strongly believe that meditation and mindful...,1
...,...,...
44201,"""Oh man I didn't make the soccer team!"", yelle...",0
44202,I believe that using this technology could be ...,0
44203,The Face on Mars is a fascinating phenomenon t...,1
44204,Texting & Driving\n\nUsing your phone while dr...,0


In [85]:
train_essays.rename(columns={'generated':'label'},inplace=True)

In [86]:
final_data = pd.concat([train_essays,new_data])

In [87]:
labels = final_data['label']
text = final_data['text']

In [76]:
labels = labels.to_numpy().reshape((-1,1))

In [96]:
df=pd.DataFrame(text)

In [110]:
df = df.astype('string').to_numpy()

In [119]:
vec_layer2 = TextVectorization(max_tokens=1000000,output_mode='int',output_sequence_length=250)
vec_layer2.adapt(df)
train = np.array(vec_layer2(df))

2023-11-23 15:31:45.851432: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [120]:
x_train,x_test,y_train,y_test = train_test_split(train,labels)

In [125]:
y_train = y_train.to_numpy().reshape((-1,1))
y_test = y_test.to_numpy().reshape((-1,1))

In [173]:
model2 = Sequential()
model2.add(Embedding(input_dim=48585,output_dim=32,input_length=250))
model2.add(LSTM(32, return_sequences=False))
model2.add(Dropout(0.5))
model2.add(Dense(1,activation='sigmoid'))


model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

history = model2.fit(x_train,y_train,epochs=10,callbacks=callbacks)

Epoch 1/10


2023-11-23 17:38:22.893700: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-23 17:38:23.047699: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-23 17:38:23.426185: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1069/1069 [==============================] - 45s 41ms/step - loss: 0.2578 - acc: 0.9080
Epoch 2/10
1069/1069 [==============================] - 38s 35ms/step - loss: 0.1367 - acc: 0.9587
Epoch 3/10
1069/1069 [==============================] - 37s 35ms/step - loss: 0.0822 - acc: 0.9790
Epoch 4/10
1069/1069 [==============================] - 36s 34ms/step - loss: 0.0462 - acc: 0.9901
Epoch 5/10
1069/1069 [==============================] - 36s 34ms/step - loss: 0.0580 - acc: 0.9870


In [168]:
# from keras.models import Sequential
# from keras.layers import Embedding, LSTM, Dropout, Flatten, Dense

# model2 = Sequential()

# # Add an Embedding layer with input_shape
# model2.add(Embedding(input_dim=48585, output_dim=32, input_length=250))

# # Add an LSTM layer with the correct input_shape and number of units
# model2.add(LSTM(32, return_sequences=True))  # Adjust return_sequences based on your model architecture

# # Add a Dropout layer with a specified dropout rate
# model2.add(Dropout(0.5))  # Adjust the dropout rate as needed

# # Add a Flatten layer
# model2.add(Flatten())

# # Add a Dense layer with sigmoid activation for binary classification
# model2.add(Dense(1, activation='sigmoid'))

# model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# # Make sure x_train and y_train have the correct shapes
# history = model2.fit(x_train, y_train, epochs=20, callbacks=callbacks)


In [175]:
model2.evaluate(x_test,y_test)

2023-11-23 17:42:04.003569: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-23 17:42:04.101534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


357/357 [==============================] - 5s 11ms/step - loss: 0.0678 - acc: 0.9827


[0.06777850538492203, 0.9827132225036621]

In [177]:
pred=model2.predict(x_test)

357/357 [==============================] - 3s 10ms/step
